## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham". 

-### Данные

Для удобства данные приложены в описании задания

In [2]:
import pandas as pd
df = pd.read_csv('../data/W8T1/3_data.csv', encoding='latin-1')

Оставляем только интересующие нас колонки — тексты смс и метки:

In [3]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [4]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [5]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [12]:
import nltk
from nltk import stem
from nltk.corpus import stopwords
import re

nltk.download("stopwords")

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess(text: str):
    text = re.sub(r'[^\w\s]', '', text) #only words left
    text = text.lower()
    text_split = text.split(' ')
    text_split = list(filter(lambda x : x not in stopwords, text_split))
    text_norm = map(lambda x : stemmer.stem(x), text_split)

    return ' '.join(text_norm)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arbyz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверка, что функция работает верно

In [13]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [14]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [15]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.2, random_state=41)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [25]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 41, C = 1.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Для самопроверки. Если вы верно дополнили функцию ```preprocess```, то должны получиться следующие результаты оценки модели.

In [26]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.977     0.994     0.986       898
           1      0.958     0.846     0.898       136

    accuracy                          0.975      1034
   macro avg      0.968     0.920     0.942      1034
weighted avg      0.975     0.975     0.974      1034



Выполним предсказание для конкретного текста

In [21]:
txt = "As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a å£1500 Bonus Prize, call 09066364589"
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [22]:
model.predict(txt)

array([1], dtype=int64)

In [ ]:
#Сообщение помечено как спам.

In [27]:
test_data = ['Get SMART Spam Control That Always Delivers The Email You Want! Finally, we discovered the ultimate solution that is guaranteed to stop all spam without losing any of your important email!', 'This is a fine collection of articles by Bernard Lewis about the Middle East.', 'Reduce your mortgage payments Interest Rates are Going Up! Give Your Family The Financial freedom They Deserve Refinance Today and SAVE', 'Take your prize, more than 100 computers, smartphones and TVs are supposed to be played in a free quiz. Call by phone 8 800 243 456']

preprocessed_test_data = list(map(lambda x: preprocess(x), test_data))
vectorized_test_data = vectorizer.transform(preprocessed_test_data)
model.predict(vectorized_test_data)

array([0, 0, 0, 1], dtype=int64)